In [ ]:
from PIL import Image
import numpy as np
import skimage
import skimage.morphology
import skimage.feature
import skan
from scipy import ndimage as ndi
from tifffile import imsave
from scipy.interpolate import interp1d


def save_image_as_tiff(image,filename):
	image = skimage.morphology.dilation(image)
	image = np.einsum('ijk->kij', image)
	image = np.flip(image, axis=(1, 2))
	imsave(filename,image)


def tubularity_detection(image_array, sigma):
	hessian = skimage.feature.hessian_matrix(image_array, sigma=sigma)
	hessian_eig = np.array(skimage.feature.hessian_matrix_eigvals(hessian))
	ra = np.absolute(hessian_eig[1])/np.absolute(hessian_eig[0])
	rb = np.absolute(hessian_eig[2])/np.sqrt(np.absolute(np.multiply(hessian_eig[1], hessian_eig[0])))
	s = np.sqrt(np.square(hessian_eig[1])+np.square(hessian_eig[2]))
	c = (np.absolute(hessian[0])+np.absolute(hessian[1])+np.absolute(hessian[2])+np.absolute(hessian[3])+
	np.absolute(hessian[4])+np.absolute(hessian[5]))/2
	c = np.square(c)*2
	tubularity = np.multiply(1-np.exp(-(np.square(ra)/0.5)), np.exp(-(np.square(rb)/0.5)))
	tubularity = np.multiply(tubularity, 1-np.exp(-np.divide(np.square(s), c)))
	return tubularity


def image_pro(image_array):
	image_array = image_array / np.amax(image_array)
	image_array = skimage.filters.gaussian(image_array, sigma=1)
	image_array = skimage.morphology.closing(image_array)

	block_size = 9
	threshold = np.zeros(image_array.shape)
	#for j in range(image_array.shape[2]):
	#	threshold[:, :, j] = skimage.filters.threshold_local(image_array[:, :, j], block_size, offset=0)
	mask = image_array > 0.15
	mask = mask | remove_hole(~mask)
	mask = skimage.morphology.closing(mask)
	mask = ndi.binary_fill_holes(mask)
	'''
	sigma_range = 29
	x, y, z = image_array.shapeinter
	tubularity = np.stack([mask, mask], axis=3)
	print tubularity.shape
	for j in range(sigma_range):
		tubularity[:, :, :, 1] = tubularity_detection(mask, i+1)
		tubularity[:, :, :, 0] = np.amax(tubularity,axis=3)
	return tubularity[:, :, :, 0]
	'''
	return mask


def remove_false_hole(binary_voxel_image):
	labelled_image = skimage.measure.label(binary_voxel_image)
	object_features = skimage.measure.regionprops(labelled_image)
	object_area = [objf["area"] for objf in object_features]
	object_bbox_area = [objf["bbox_area"] for objf in object_features]
	object_bbox = [objf["bbox"] for objf in object_features]
	object_coordinate = [objf["coords"] for objf in object_features]
	small_object = np.array([[], [], []])
	small_object = np.einsum('ij->ji', small_object)
	for j in range(len(object_bbox)):
		bbox = object_bbox[j]
		if ((bbox[5]-bbox[2]) < 5) | (object_area[j] < object_bbox_area[j]/6):
			small_object = np.concatenate([small_object, object_coordinate[j]])
	small_object = np.array(small_object, dtype=int)
	l, ll = small_object.shape
	for j in range(l):
		binary_voxel_image[small_object[j, 0], small_object[j, 1], small_object[j, 2]] = False
	return binary_voxel_image


def remove_hole(binary_voxel_image):
	fill = np.zeros(binary_voxel_image.shape, dtype=bool)
	for k in range(binary_voxel_image.shape[2]):
		binary_image = binary_voxel_image[:, :, k]
		labelled_image = skimage.measure.label(binary_image)
		object_features = skimage.measure.regionprops(labelled_image)
		object_area = [objf["area"] for objf in object_features]
		object_coordinate = [objf["coords"] for objf in object_features]
		small_object = np.array([[], []])
		small_object = np.einsum('ij->ji', small_object)
		for j in range(len(object_area)):
			if object_area[j] < 20000:
				small_object = np.concatenate([small_object, object_coordinate[j]])
		small_object = np.array(small_object, dtype=int)
		l, ll = small_object.shape
		for j in range(l):
			fill[small_object[j, 0], small_object[j, 1], k] = True
	return remove_false_hole(fill)


def remove_hole2(binary_voxel_image):
	fill = np.zeros(binary_voxel_image.shape, dtype=bool)
	labelled_image = skimage.measure.label(binary_voxel_image)
	object_features = skimage.measure.regionprops(labelled_image)
	object_area = [objf["area"] for objf in object_features]
	object_coordinate = [objf["coords"] for objf in object_features]
	if len(object_area) > 1:
		object_coordinate.pop(object_area.index(max(object_area)))
		if len(object_area) > 2:
			small_object = np.concatenate(object_coordinate, axis=0)
		else:
			small_object = object_coordinate[0]
		print (small_object.shape)
		l, ll = small_object.shape
		for j in range(l):
			fill[small_object[j, 0], small_object[j, 1], small_object[j, 2]] = True
	return fill


def obtain_skeleton(binary_voxel_image):
	skeleton = medial_axis_3d(binary_voxel_image)
	labelled_image = skimage.measure.label(skeleton)
	object_features = skimage.measure.regionprops(labelled_image)
	object_area = [objf["area"] for objf in object_features]
	object_coordinate = [objf["coords"] for objf in object_features]
	max_skeleton = np.zeros(skeleton.shape)
	max_skeleton_coordinate = object_coordinate[object_area.index(max(object_area))]
	l, ll = max_skeleton_coordinate.shape
	for j in range(l):
		max_skeleton[max_skeleton_coordinate[j, 0], max_skeleton_coordinate[j, 1], max_skeleton_coordinate[j, 2]] = 225
	save_image_as_tiff(max_skeleton,'C:/Users/brian/Documents/skeleton.tif')
	return max_skeleton


def pruning(binary_voxel_image):
	x, y, z = np.shape(binary_voxel_image)
	original_image = binary_voxel_image
	binary_voxel_image = np.array(dtype=uint8)
	binary_voxel_image = np.pad(binary_voxel_image, (1, 1), mode='constant', constant_values=1)
	neighbor=binary_voxel_image_padded[:-2, 1:-1, 1:-1] + binary_voxel_image[:-2, :-2, 1:-1] + binary_voxel_image[:-2, 2:, 1:-1] \
	         + binary_voxel_image[:-2, 1:-1, 2:] + binary_voxel_image[:-2, :-2, 2:] + binary_voxel_image[:-2, 2:, 2:] \
	         + binary_voxel_image[:-2, 1:-1, :-2] + binary_voxel_image[:-2, :-2, :-2] + binary_voxel_image[:-2, 2:, :-2] \
	         + binary_voxel_image[2:, 1:-1, :-2] + binary_voxel_image[2:, :-2, :-2] + binary_voxel_image[2:, 2:, :-2] \
	         + binary_voxel_image[2:, 1:-1, 1:-1] + binary_voxel_image[2:, :-2, 1:-1] + binary_voxel_image[2:, 2:, 1:-1] \
	         + binary_voxel_image[2:, 1:-1, 2:] + binary_voxel_image[2:, :-2, 2:] + binary_voxel_image[2:, 2:, 2:] \
	         + binary_voxel_image[1:-1, 1:-1, :-2] + binary_voxel_image[1:-1, :-2, :-2] + binary_voxel_image[1:-1, 2:, :-2] \
	         + binary_voxel_image[1:-1, :-2, 1:-1] + binary_voxel_image[1:-1, 2:, 1:-1] \
	         + binary_voxel_image[1:-1, 1:-1, 2:] + binary_voxel_image[1:-1, :-2, 2:] + binary_voxel_image[1:-1, 2:, 2:] \

	def trace_dendrite(x,y,z):
		while neighbor<=2:
			a=1
	for i in range(x-1):
		for j in range(y-1):
			for k in range(z-1):
				if original_image[i, j, k] & neighbor[x,y,z]==1:
					trace_dendrite()
				break

In [ ]:
img = Image.open('C:/Users/brian/Documents/nTracer sample.tif')
h, w = np.shape(img)
nframes = int(img.n_frames/4)
img_r = np.zeros((h, w, nframes))
img_g = np.zeros((h, w, nframes))
img_b = np.zeros((h, w, nframes))
img_a = np.zeros((h, w, nframes))
for i in range(nframes - 1):
	img.seek(i*4+1)
	img_r[:, :, i] = np.array(img)
	img.seek(i * 4 + 2)
	img_g[:, :, i] = np.array(img)
	img.seek(i * 4 + 3)
	img_b[:, :, i] = np.array(img)
	img.seek(i * 4 + 4)
	img_a[:, :, i] = np.array(img)
img_rgba = np.max(np.stack([img_r, img_g, img_b, img_a]), axis=0)
#f_interpolation = interp1d(np.linspace(0, nframes-1, nframes), img_rgba, axis=2)
#img_rgba = f_interpolation(np.linspace(0, nframes-1, 2*nframes-1))

new_image = image_pro(img_rgba)
